<a href="https://colab.research.google.com/github/seanmcalevey/example-repo/blob/master/keras_mlp_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
def keras_mlp(train_matrix, val_matrix, y_train, y_val, hidden_layers=3,
              neuron_layers=(128, 128, 128), dropout_layers=(None, None, None),
              epochs=20, batch_size=32, patience=2, activation='relu', optimizer='adam',
              batch_norm=True, restore_best_weights=True):

  # Manage warnings:

  import warnings

  warnings.filterwarnings('ignore')
  
  # Import relevant modules

  import numpy as np
  import pandas as pd
  from keras.models import Sequential
  from keras.layers import Dense, Dropout, BatchNormalization
  from sklearn.metrics import classification_report, f1_score
  from keras.callbacks import EarlyStopping

  if hidden_layers == 4:
    
    neurons_1, neurons_2, neurons_3, neurons_4 = neuron_layers[0], neuron_layers[1], neuron_layers[2], neuron_layers[3]

    dropout_1, dropout_2, dropout_3, dropout_4 = dropout_layers[0], dropout_layers[1], dropout_layers[2], dropout_layers[3]

  elif hidden_layers == 3:

    # Unpack hyperparameters:

    neurons_1, neurons_2, neurons_3 = neuron_layers[0], neuron_layers[1], neuron_layers[2]

    dropout_1, dropout_2, dropout_3 = dropout_layers[0], dropout_layers[1], dropout_layers[2]
  
  elif hidden_layers == 2:

    neurons_1, neurons_2 = neuron_layers[0], neuron_layers[1]

    dropout_1, dropout_2 = dropout_layers[0], dropout_layers[1]
  
  elif hidden_layers == 1:

    neurons_1 = neuron_layers[0]

    dropout_1 = dropout_layers[0]

  # Set input dimensions

  input_dim = train_matrix.shape[1]

  # Build model

  model = Sequential()

  # Layer 1
  model.add(Dense(neurons_1, input_dim=input_dim, activation=activation))
  if dropout_1 != None:
    model.add(Dropout(dropout_1))
  if batch_norm == True:
    model.add(BatchNormalization())

  # Layer 2 (Optional)
  if hidden_layers >= 2:

    model.add(Dense(neurons_2, activation=activation))
    if dropout_2 != None:
      model.add(Dropout(dropout_2))
    if batch_norm == True:
      model.add(BatchNormalization())

  # Layer 3 (Optional)
  if hidden_layers >= 3:
  
    model.add(Dense(neurons_3, activation=activation))
    if dropout_3 != None:
      model.add(Dropout(dropout_3))
    if batch_norm == True:
      model.add(BatchNormalization())
  
  # Layer 4 (Optional)
  if hidden_layers >= 4:

    model.add(Dense(neurons_4, activation=activation))
    if dropout_4 != None:
      model.add(Dropout(dropout_4))
    if batch_norm == True:
      model.add(BatchNormalization())

  # Output Layer
  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  # Fit and Score Model

  callback = EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, restore_best_weights=restore_best_weights)

  model_instance = model.fit(train_matrix, y_train, batch_size=batch_size, epochs=epochs, callbacks=[callback], validation_split=0.2)

  # Small function to binarize predicted outputs from model

  def binizer(x):
    if x > 0.5:
      return 1
    else:
      return 0

  # Get train score

  y_train_pred = model.predict(train_matrix)

  y_train_pred_bin = pd.Series(list(y_train_pred.reshape(-1,1))).apply(lambda x: binizer(x))

  train_f1 = f1_score(y_train, y_train_pred_bin)

  train_score = model.evaluate(train_matrix, y_train)[1]

  # Get validation score

  y_val_pred = model.predict(val_matrix)

  y_val_pred_bin = pd.Series(list(y_val_pred.reshape(-1,1))).apply(lambda x: binizer(x))

  val_f1 = f1_score(y_val, y_val_pred_bin)

  val_score = model.evaluate(val_matrix, y_val)[1]

  return model_instance, train_score, train_f1, val_score, val_f1

In [0]:
def keras_mlp_generator(generator, val_generator, train_matrix, val_matrix, hidden_layers=3, neuron_layers=(128, 128, 128),
                        dropout_layers=(None, None, None), epochs=20, batch_size=32, patience=2, activation='relu', batch_norm=True,
                        restore_best_weights=True, learning_rate=0.01):
  
  import math

  # Manage warnings:

  import warnings

  warnings.filterwarnings('ignore')
  
  # Import relevant modules

  import numpy as np
  import pandas as pd
  from keras.models import Sequential
  from keras.layers import Dense, Dropout, BatchNormalization
  from sklearn.metrics import classification_report, f1_score
  from keras.callbacks import EarlyStopping
  from keras.optimizers import Adam

  if hidden_layers == 4:
    
    neurons_1, neurons_2, neurons_3, neurons_4 = neuron_layers[0], neuron_layers[1], neuron_layers[2], neuron_layers[3]

    dropout_1, dropout_2, dropout_3, dropout_4 = dropout_layers[0], dropout_layers[1], dropout_layers[2], dropout_layers[3]

  elif hidden_layers == 3:

    # Unpack hyperparameters:

    neurons_1, neurons_2, neurons_3 = neuron_layers[0], neuron_layers[1], neuron_layers[2]

    dropout_1, dropout_2, dropout_3 = dropout_layers[0], dropout_layers[1], dropout_layers[2]
  
  elif hidden_layers == 2:

    neurons_1, neurons_2 = neuron_layers[0], neuron_layers[1]

    dropout_1, dropout_2 = dropout_layers[0], dropout_layers[1]
  
  elif hidden_layers == 1:

    neurons_1 = neuron_layers[0]

    dropout_1 = dropout_layers[0]

  # Set input dimensions

  input_dim = train_matrix.shape[1]

  # Build model

  model = Sequential()

  # Layer 1
  model.add(Dense(neurons_1, input_dim=input_dim, activation=activation))
  if dropout_1 != None:
    model.add(Dropout(dropout_1))
  if batch_norm == True:
    model.add(BatchNormalization())

  # Layer 2 (Optional)
  if hidden_layers >= 2:

    model.add(Dense(neurons_2, activation=activation))
    if dropout_2 != None:
      model.add(Dropout(dropout_2))
    if batch_norm == True:
      model.add(BatchNormalization())

  # Layer 3 (Optional)
  if hidden_layers >= 3:
  
    model.add(Dense(neurons_3, activation=activation))
    if dropout_3 != None:
      model.add(Dropout(dropout_3))
    if batch_norm == True:
      model.add(BatchNormalization())
  
  # Layer 4 (Optional)
  if hidden_layers >= 4:

    model.add(Dense(neurons_4, activation=activation))
    if dropout_4 != None:
      model.add(Dropout(dropout_4))
    if batch_norm == True:
      model.add(BatchNormalization())

  # Output Layer
  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate), metrics=['accuracy'])

  # Fit and Score Model

  callback = EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, restore_best_weights=restore_best_weights)

  model_instance = model.fit_generator(generator, steps_per_epoch=math.ceil(train_matrix.shape[0]/batch_size), epochs=epochs, callbacks=[callback],
                                       validation_data=val_generator, validation_steps=math.ceil(val_matrix.shape[0]/batch_size))

  # Small function to binarize predicted outputs from model

  def binizer(x):
    if x > 0.5:
      return 1
    else:
      return 0

  # Get train score

  y_train_pred = model.predict(train_matrix)

  y_train_pred_bin = pd.Series(list(y_train_pred.reshape(-1,1))).apply(lambda x: binizer(x))

  train_f1 = f1_score(y_train, y_train_pred_bin)

  train_score = model.evaluate(train_matrix, y_train)[1]

  # Get validation score

  y_val_pred = model.predict(val_matrix)

  y_val_pred_bin = pd.Series(list(y_val_pred.reshape(-1,1))).apply(lambda x: binizer(x))

  val_f1 = f1_score(y_val, y_val_pred_bin)

  val_score = model.evaluate(val_matrix, y_val)[1]

  return model_instance, train_score, train_f1, val_score, val_f1